In [1]:
import numpy as np
from scipy.optimize import minimize
from scipy.stats import t

def fit_t_regression(x1, x2, x3, y):
    x1, x2, x3, y = map(lambda v: np.asarray(v).ravel(), (x1, x2, x3, y))
    mask = np.isfinite(x1) & np.isfinite(x2) & np.isfinite(x3) & np.isfinite(y)
    x1, x2, x3, y = x1[mask], x2[mask], x3[mask], y[mask]
    n = y.size
    X = np.column_stack([np.ones(n), x1, x2, x3])
    beta0, *_ = np.linalg.lstsq(X, y, rcond=None)
    resid0 = y - X @ beta0
    scale0 = float(np.std(resid0, ddof=0)) or 1.0
    init = np.hstack([10.0, beta0, scale0])
    def negloglike(params):
        df, beta, sc = params[0], params[1:-1], params[-1]
        if df <= 1.0 or sc <= 0.0:
            return np.inf
        mu_i = X @ beta
        return -np.sum(t.logpdf(y, df, loc=mu_i, scale=sc))
    bounds = [(1.01, 200.0)] + [(None, None)]*4 + [(1e-8, None)]
    res = minimize(negloglike, init, method="L-BFGS-B", bounds=bounds)
    df_hat   = float(res.x[0])
    Alpha    = float(res.x[1])
    B1, B2, B3 = map(float, res.x[2:5])
    sigma    = float(res.x[-1])
    mu_hat   = float(np.mean(X @ np.array([Alpha, B1, B2, B3])))
    return mu_hat, sigma, df_hat, Alpha, B1, B2, B3

In [2]:
import pandas as pd

data = pd.read_csv('../testfiles/data/test7_3.csv')
mu, sigma, nu, Alpha, B1, B2, B3 = fit_t_regression(
    data['x1'], data['x2'], data['x3'], data['y']
)

print("Estimated mu =", mu)
print("Estimated sigma =", sigma)
print("Estimated nu =", nu)
print("Estimated Alpha =", Alpha)
print("Estimated B1 =", B1)
print("Estimated B2 =", B2)
print("Estimated B3 =", B3)

Estimated mu = 0.02277627711878503
Estimated sigma = 0.048548709805701556
Estimated nu = 4.598341945488593
Estimated Alpha = 0.04263498195098822
Estimated B1 = 0.9748829859544208
Estimated B2 = 2.041207299427949
Estimated B3 = 3.1547914215993105
